In [4]:
# decision_tree_learning
import numpy as np
from typing import Tuple

# Load datasets from files
clean_dataset = np.loadtxt("wifi_db/clean_dataset.txt", delimiter="\t")
noisy_dataset = np.loadtxt("wifi_db/noisy_dataset.txt", delimiter=" ")
print(clean_dataset)
print(noisy_dataset)

SIGNAL_COUNT = 7
CLASS_COUNT = 4

def all_same_label(dataset) -> bool:
  # check if all samples have the same label
  for i in range(len(dataset)):
    if dataset[i][SIGNAL_COUNT] != dataset[0][SIGNAL_COUNT]:
      return False
  return True

def entropy(dataset) -> float:
  # H(D)
  
  # To implement

  return 0.0

def calculate_IG(l_dataset, r_dataset) -> float:
  # IG(D1, D2) = H(D) - H(D1, D2)
  # H(D1, D2) = (|D1|/|D|)H(D1) + (|D2|/|D|)H(D2)

  # To implement

  return 0.0

def pick_split_value(dataset, attribute) -> Tuple[float, float]:
  # find the best value to split by
  highest_IG = 0.0
  best_value = 0.0

  np.argsort(dataset[:,attribute])

  for i in range(len(dataset)):
    l_dataset = dataset[:i]
    r_dataset = dataset[i:]

    information_gain = calculate_IG(l_dataset, r_dataset)

    if information_gain > highest_IG:
      highest_IG = information_gain
      best_value = dataset[i][attribute]
  
  return (highest_IG, best_value)

def find_split(dataset):
  # find the best split
  best_attribute = 0
  best_value = 0.0
  best_IG = 0.0

  for i in range(SIGNAL_COUNT):
    (information_gain, value) = pick_split_value(dataset, i)

    if information_gain > best_IG:
      best_IG = information_gain
      best_attribute = i
      best_value = value
  
  return (best_attribute, best_value)

def decision_tree_learning(dataset, depth) -> Tuple[dict, int]:
  if all_same_label(dataset):
    return ({"value": dataset[0][SIGNAL_COUNT]}, depth)
  else:
    (split_attribute, split_value) = find_split(dataset)

    l_dataset = dataset[dataset[:,split_attribute] < split_value]
    r_dataset = dataset[dataset[:,split_attribute] >= split_value]

    node = {"split_attribute": split_attribute, "split_value": split_value}

    l_branch, l_depth = decision_tree_learning(l_dataset, depth+1)
    r_branch, r_depth = decision_tree_learning(r_dataset, depth+1)

    node["l_branch"] = l_branch
    node["r_branch"] = r_branch

    return (node, max(l_depth, r_depth))

# print(decision_tree_learning(clean_dataset, 0))


[[-64. -56. -61. ... -82. -81.   1.]
 [-68. -57. -61. ... -85. -85.   1.]
 [-63. -60. -60. ... -85. -84.   1.]
 ...
 [-62. -59. -46. ... -87. -88.   4.]
 [-62. -58. -52. ... -90. -85.   4.]
 [-59. -50. -45. ... -88. -87.   4.]]
[[-59. -53. -51. ... -79. -87.   4.]
 [-66. -53. -59. ... -81. -79.   1.]
 [-41. -57. -63. ... -66. -65.   2.]
 ...
 [-57. -54. -56. ... -79. -82.   1.]
 [-56. -52. -50. ... -85. -88.   3.]
 [-46. -54. -47. ... -80. -73.   3.]]


In [ ]:
# tree visualiser

In [ ]:
# evaluation function

In [ ]:
# classification metrics generator